In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tc = sns.load_dataset("titanic")

%matplotlib inline

In [2]:
fact_table = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Fact_table', engine='openpyxl')
trans_dim = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Trans_dim', engine='openpyxl')
item_dim = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Item_dim', engine='openpyxl')
customer_dim = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Customer_dim', engine='openpyxl')
time_dim = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Time_dim', engine='openpyxl')
store_dim = pd.read_excel(r'e-com-sales-out.xlsx', sheet_name='Store_dim', engine='openpyxl')

In [3]:
fact_table

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P025,C001495,T08774,I00185,S0038,19,ct,14.00,266.0
1,P009,C000904,T09092,I00017,S0007,8,cans,6.75,54.0
2,P019,C001627,T03980,I00235,S0013,1,ct,14.00,14.0
3,P004,C001606,T00982,I00232,S0038,7,ct,12.00,84.0
4,P019,C001273,T09057,I00075,S0012,35,cans,15.50,542.5
...,...,...,...,...,...,...,...,...,...
9995,P007,C001501,T06807,I00028,S0007,10,cans,6.75,67.5
9996,P004,C000405,T02560,I00143,S0031,43,ct,16.00,688.0
9997,P015,C001789,T07589,I00142,S0021,152,ct,12.50,1900.0
9998,P003,C001500,T00881,I00194,S0037,19,bars,23.00,437.0


In [4]:
!pip install psycopg2

In [5]:
import psycopg2
import psycopg2.extras

In [6]:
item_dim.dtypes

item_key           object
item_name          object
description        object
unit_price        float64
stock_quantity      int64
unit               object
dtype: object

In [7]:
item_dim['unit_price'] = pd.to_numeric(item_dim['unit_price'])

In [8]:
item_dim.dtypes

item_key           object
item_name          object
description        object
unit_price        float64
stock_quantity      int64
unit               object
dtype: object

In [9]:
time_dim.dtypes

time_key    object
date        object
hour         int64
day          int64
week        object
month        int64
quarter     object
year         int64
dtype: object

In [10]:
time_dim['date'] = pd.to_datetime(time_dim['date'], format="mixed")
time_dim

,time_key,date,hour,day,week,month,quarter,year
0,T00001,2015-08-06 13:53:00,13,8,2nd Week,6,Q2,2015
1,T00002,2014-06-30 07:44:00,7,30,4th Week,6,Q2,2014
2,T00003,2014-03-11 05:32:00,5,3,1st Week,11,Q4,2014
3,T00004,2018-09-10 16:09:00,16,9,2nd Week,10,Q4,2018
4,T00005,2020-05-25 08:32:00,8,25,4th Week,5,Q2,2020
...,...,...,...,...,...,...,...,...
9994,T09995,2022-12-09 17:55:00,17,12,2nd Week,9,Q3,2022
9995,T09996,2020-07-28 16:39:00,16,28,4th Week,7,Q3,2020
9996,T09997,2017-04-07 12:28:00,12,4,1st Week,7,Q3,2017
9997,T09998,2022-02-28 22:42:00,22,28,4th Week,2,Q1,2022


In [11]:
time_dim.dtypes

time_key            object
date        datetime64[ns]
hour                 int64
day                  int64
week                object
month                int64
quarter             object
year                 int64
dtype: object

In [12]:
fact_table.dtypes

payment_key       object
coustomer_key     object
time_key          object
item_key          object
store_key         object
quantity           int64
unit              object
unit_price       float64
total_price      float64
dtype: object

In [13]:
fact_table['unit_price'] = pd.to_numeric(fact_table['unit_price'])
fact_table['total_price'] = pd.to_numeric(fact_table['total_price'])
fact_table.dtypes

payment_key       object
coustomer_key     object
time_key          object
item_key          object
store_key         object
quantity           int64
unit              object
unit_price       float64
total_price      float64
dtype: object

In [14]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "1234",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("Connection to DB established!")
        return self.connection

con = PostgresConnection().getConnection()

Connection to DB established!


In [15]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.trans_dim(payment_key, trans_type, bank_name)" \
                "VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, trans_dim.values)
con.commit()
cur.close()

Connection to DB established!


In [16]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.item_dim(item_key,item_name,description,unit_price,stock_quantity,unit)" \
                "VALUES (%s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, item_dim.values)
con.commit()
cur.close()

Connection to DB established!


In [18]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.customer_dim(coustomer_key,name,contact_no,nid,address,street,upazila,district,division)" \
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, customer_dim.values)
con.commit()
cur.close()

Connection to DB established!


In [19]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.time_dim(time_key,date,hour,day,week,month,quarter,year)" \
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, time_dim.values)
con.commit()
cur.close()

Connection to DB established!


In [21]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.store_dim(store_key,store_size,location,city,upazila,district,division)" \
                "VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, store_dim.values)
con.commit()
cur.close()

Connection to DB established!


In [22]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO ecomdb_star_schema.fact_table(payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price)" \
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, fact_table.values)
con.commit()
cur.close()

Connection to DB established!
